# Investigation
I wonder where the other languages have gone? In the original items there were how many languages? Why do some languages have so few segments?

In [1]:
import pandas as pd
import numpy as np
import os
import sys
module_path = os.path.abspath(os.path.join('../vad_utils'))
if module_path not in sys.path:
    sys.path.append(module_path)
from vad_utils import SAMPLING_RATE, FRAME_SIZE_MS, SAMPLES_PER_FRAME
import vad_utils as vu
from pydub import AudioSegment
import pickle as pkl
from pathlib import Path
import time
import glob


In [2]:
# Now read in the description of the input and remove the unwanted columns and rename the rest to be python attribute names.
items = pd.read_csv('/home/jovyan/work/GRN-Notebooks/Data/all_items_for_processing.csv')
items = items.drop(columns=['Unnamed: 0', 'LanguageID', 'Language', 'Track', 'Recordist', 'Size', 'Length', 'Tape Side', 'Item Type', 'composite'])
items.rename(inplace=True, columns={ 'ISO' : 'iso', 'Location' : 'location', 'Year' : 'year', 'Path' : 'path', 'Filename' : 'filename', 'Title' : 'title', 'Program Item Number' : 'item_no', 'Program' : 'program' })

# the existing ID is based on track number rather than item number. We want to use the item number.
items['ID'] = items['program'] + '_' + items['item_no'].astype(int).apply('{:0>3d}'.format)

items.set_index('ID', inplace=True)
items.columns


Index(['iso', 'program', 'location', 'year', 'path', 'filename', 'item_no',
       'title', 'start', 'end'],
      dtype='object')

In [3]:
# the duplicates appear to be caused by items that have been incorrectly numbered. Take the
# last of each. 
items1 = items[items.index.duplicated('last') == False]

How many languages in this set?

In [6]:
langs = items1['iso'].value_counts()
len(langs)


3267

In [5]:
langs1 = items['iso'].value_counts()